In [2]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math

In [2]:
class WineDataset(Dataset):
    
    def __init__(self):
        # data loading
        xy = np.loadtxt('./wine.csv', delimiter=",", dtype=np.float32, skiprows=1)
        self.x = torch.from_numpy(xy[:,1:])
        self.y = torch.from_numpy(xy[:,[0]]) #n_samples, 1
        self.n_samples = xy.shape[0]
        
    def __getitem__(self,index):
        return self.x[index], self.y[index]
        
    def __len__(self):
        return self.n_samples

In [3]:
dataset = WineDataset()

In [4]:
first_data = dataset[0]
dataloader = DataLoader(dataset=dataset,batch_size=4,shuffle=True,num_workers=2)

In [5]:
dataiter = iter(dataloader)
data = dataiter.next()

In [6]:
num_epochs = 2
batch_size = 4
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/batch_size)
print(total_samples,n_iterations)

178 45


In [7]:
# torchvision.datasets.MNIST(), fashion-mnist-cifar-coco
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(dataloader):
        if (i+1)%5 ==0:
            print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_iterations}, inputs {inputs.shape}')

epoch 1/2, step 5/45, inputs torch.Size([4, 13])
epoch 1/2, step 10/45, inputs torch.Size([4, 13])
epoch 1/2, step 15/45, inputs torch.Size([4, 13])
epoch 1/2, step 20/45, inputs torch.Size([4, 13])
epoch 1/2, step 25/45, inputs torch.Size([4, 13])
epoch 1/2, step 30/45, inputs torch.Size([4, 13])
epoch 1/2, step 35/45, inputs torch.Size([4, 13])
epoch 1/2, step 40/45, inputs torch.Size([4, 13])
epoch 1/2, step 45/45, inputs torch.Size([2, 13])
epoch 2/2, step 5/45, inputs torch.Size([4, 13])
epoch 2/2, step 10/45, inputs torch.Size([4, 13])
epoch 2/2, step 15/45, inputs torch.Size([4, 13])
epoch 2/2, step 20/45, inputs torch.Size([4, 13])
epoch 2/2, step 25/45, inputs torch.Size([4, 13])
epoch 2/2, step 30/45, inputs torch.Size([4, 13])
epoch 2/2, step 35/45, inputs torch.Size([4, 13])
epoch 2/2, step 40/45, inputs torch.Size([4, 13])
epoch 2/2, step 45/45, inputs torch.Size([2, 13])


In [13]:
class WineDataset(Dataset):
    
    def __init__(self,transform=None):
        # data loading
        xy = np.loadtxt('./wine.csv', delimiter=",", dtype=np.float32, skiprows=1)
        self.n_samples = xy.shape[0]
        
        self.x = xy[:,1:]
        self.y = xy[:,[0]] #n_samples, 1
        
        self.transform = transform
        
    def __getitem__(self, index):
        sample = self.x[index], self.y[index]
    
        if self.transform:
            sample = self.transform(sample)
            
        return sample
        
    def __len__(self):
        return self.n_samples

In [14]:
class ToTensor():
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)
    
class MulTransform():
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, sample):
        inputs, targets = sample
        inputs *= self.factor
        return inputs, targets   

In [15]:
dataset = WineDataset(transform=ToTensor())
first_data = dataset[0]
print(first_data)

(tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03]), tensor([1.]))


In [16]:
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(4)])
dataset = WineDataset(transform=composed)
first_data = dataset[0]
print(first_data)

(tensor([5.6920e+01, 6.8400e+00, 9.7200e+00, 6.2400e+01, 5.0800e+02, 1.1200e+01,
        1.2240e+01, 1.1200e+00, 9.1600e+00, 2.2560e+01, 4.1600e+00, 1.5680e+01,
        4.2600e+03]), tensor([1.]))


In [22]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x),axis=0)

def cross_entropy(actual, predicted):
    loss = - np.sum(actual*np.log(predicted))
    return loss # /float(predicted.shape[0])

In [4]:
x = np.array([2.0, 1.0, 0.1])
outputs = softmax(x)
print(outputs)

[0.65900114 0.24243297 0.09856589]


In [20]:
y = torch.tensor([2.0, 1.0, 0.1])
outputs = torch.softmax(y, dim=0)
print(outputs)

tensor([0.6590, 0.2424, 0.0986])


In [33]:
Y = np.array([1, 0, 0])
Y_pred = np.array([0.7, 0.2, 0.1])
l= cross_entropy(Y, Y_pred)
print(l)

0.35667494393873245


In [37]:
loss = nn.CrossEntropyLoss() # nn.LogSoftmax + nn.NLLLoss (negative log likelihood loss)

In [49]:
Y = torch.tensor([0])
Y_pred = torch.tensor([[0.7, 0.2, 0.1]])
_,predictions = torch.max(Y_pred,1)
predictions

tensor([0])

In [48]:
l1 = loss(Y_pred,Y)
l1.item()

0.7679495811462402

In [50]:
Y = torch.tensor([2,0,1])

In [57]:
Y_pred = torch.tensor([[0.7, 0.2, 0.1],[0.1, 0.8, 0.1],[0.0, 0.1, 0.9]])

In [58]:
_,predictions = torch.max(Y_pred,1)
predictions

tensor([0, 1, 2])

In [59]:
l1 = loss(Y_pred,Y)
l1.item()

1.392014980316162

In [60]:
criterion = nn.BCELoss()